# Querying Data Modeling

This tutorial demonstrates how to query a data model in Cognite Data Fuction (CDF) with Python 

**Prerequisites**

* Installed `cognite-sdk`
* Setup a service principal to connect to CDF.
* Use `cognite-toolkit` quickstart to setup configuration and populated the CogniteIndustrialData Model [Toolkit quickstart](https://docs.cognite.com/cdf/deploy/cdf_toolkit/references/packages/quickstart)

## Setting up a cognite client

In [2]:
import os

from cognite.client import CogniteClient
from dotenv import load_dotenv

load_dotenv("../.env")

client = CogniteClient.default_oauth_client_credentials(
    project=os.environ["CDF_PROJECT"],
    cdf_cluster=os.environ["CDF_CLUSTER"],
    tenant_id=os.environ["IDP_TENANT_ID"],
    client_id=os.environ["IDP_CLIENT_ID"],
    client_secret=os.environ["IDP_CLIENT_SECRET"],
)

## Querying across a direct relation

First, we will query across a direct relation. For example, we can get the parent starting from `CogniteAsset`

In [3]:
from cognite.client import data_modeling as dm
from cognite.client.data_classes import filters

asset_view = dm.ViewId("cdf_cdm", "CogniteAsset", "v1")

query = dm.query.Query(
    with_={
        "asset": dm.query.NodeResultSetExpression(limit=10, filter=filters.HasData(views=[asset_view])),
        "parent": dm.query.NodeResultSetExpression(
            from_="asset", through=dm.PropertyId(source=asset_view, property="source")
        ),
    },
    select={
        "asset": dm.query.Select(sources=[dm.query.SourceSelector(source=asset_view, properties=["*"])]),
        "parent": dm.query.Select(sources=[dm.query.SourceSelector(source=asset_view, properties=["*"])]),
    },
)

result = client.data_modeling.instances.query(query)

In [4]:
result["asset"].to_pandas(expand_properties=True)

,space,external_id,version,last_updated_time,created_time,instance_type,path,root,pathLastUpdatedTime,name,description,sourceId,sourceUpdatedUser
0,springfield_sap,WMT:23-1ST STAGE COMPRESSOR-PH,8,2024-11-08 10:29:13.060,2024-10-20 14:26:20.135,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,23-1ST STAGE COMPRESSOR-PH,1ST STAGE COMPRESSOR ON PH,681824,8137
1,springfield_sap,WMT:23-1ST STAGE COMP LUBE OIL SYS-PH,8,2024-11-08 10:29:13.060,2024-10-20 14:26:20.135,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,23-1ST STAGE COMP LUBE OIL SYS-PH,1ST STAGE COMP LUBE OIL SYS ON PH,682544,8137
2,springfield_sap,WMT:23-1ST STAGE COMP ENCLOSURE-PH,8,2024-11-08 10:29:13.060,2024-10-20 14:26:20.135,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,23-1ST STAGE COMP ENCLOSURE-PH,1ST STAGE COMP ENCLOSURE ON PH,682542,8137
3,springfield_sap,WMT:23-1ST STAGE COMPRESSION-PH,8,2024-11-08 10:29:13.060,2024-10-20 14:26:20.135,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,23-1ST STAGE COMPRESSION-PH,1ST STAGE COMPRESSION ON PH,681792,8137
4,springfield_sap,WMT:23-2ND STAGE COMPRESSOR-PH,8,2024-11-08 10:29:13.060,2024-10-20 14:26:21.187,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,23-2ND STAGE COMPRESSOR-PH,2ND STAGE COMPRESSOR ON PH,681825,8137
5,springfield_sap,WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH,8,2024-11-08 10:29:13.060,2024-10-20 14:26:21.969,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,23-1ST STAGE COMP DRY GAS SEAL SYS-PH,1ST STAGE COMP DRY GAS SEAL SYS ON PH,682540,8137
6,springfield_sap,WMT:23-2ND STAGE COMPRESSION-PH,8,2024-11-08 10:29:13.060,2024-10-20 14:26:21.969,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,23-2ND STAGE COMPRESSION-PH,2ND STAGE COMPRESSION ON PH,681794,8137
7,springfield_sap,WMT:23-2ND STAGE COMP DRY GAS SEAL SYS-PH,8,2024-11-08 10:29:13.060,2024-10-20 14:26:32.230,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,23-2ND STAGE COMP DRY GAS SEAL SYS-PH,2ND STAGE COMP DRY GAS SEAL SYS ON PH,682546,8137
8,springfield_sap,WMT:230900,8,2024-11-08 10:29:13.060,2024-10-20 14:26:32.230,node,"[{'space': 'springfield_sap', 'externalId': 'W...","{'space': 'springfield_sap', 'externalId': 'WM...",2024-11-08T10:29:13.060725+00:00,230900,GAS COMPRESSION AND RE-INJECTION ON PH,681760,8137
9,springfield_instances,WMT:23-2ND STAGE COMP DRY GAS SEAL SYS-PH,2,2024-11-08 14:20:21.633,2024-11-08 14:20:20.316,node,"[{'space': 'springfield_instances', 'externalI...","{'space': 'springfield_instances', 'externalId...",2024-11-08T14:20:21.633609+00:00,23-2ND STAGE COMP DRY GAS SEAL SYS-PH,2ND STAGE COMP DRY GAS SEAL SYS ON PH,682546,8137


In [5]:
result["parent"]

""


## Quering across reverse direct relation

## Quering across edge